In [1]:
import time
import os
import pandas as pd
import numpy as np
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
data_path="../../data/"
output_path="../../outputs/"

In [3]:
test_set_complete=pd.read_csv(output_path+"general/test_set_final.csv")
train_set=pd.read_csv(output_path+"general/train_set_final.csv")
dev_set=pd.read_csv(output_path+"general/dev_set_final.csv")
df=pd.concat([test_set_complete,dev_set,train_set]).reset_index(drop=True)

In [4]:
df=df.assign(abstract_for_prompt=np.where(df.abstract_clean!="",
                                                   "Abstract: "+df.abstract_clean,""))

df=df.assign(acknowledgments_for_prompt=np.where(df.acknowledgments_clean!="",
                                                   "\nAcknowledgments: "+df.acknowledgments_clean,""))

In [5]:
df=df.assign(abstract_for_prompt=df.abstract_for_prompt.fillna(""))

df=df.assign(paper_text="Title: "+df.title_clean+"\n"+df.abstract_for_prompt)

In [6]:
ack_preprompt="""Identify the NLP methods employed in this paper. Answer in the format [method1,...,methodn] Answer up to 3 methods."""

ack_postprompt="""What are the methods employed to solve the tasks? Don't include tasks in the answer. The NLP methods used in this paper are:\n["""

In [7]:
df=df.assign(method_prompt_text=ack_preprompt+"\n"+df.paper_text+"\n"+ack_postprompt)

## cost estimation

In [8]:
np.sum(df['task_prompt_text'].str.split().apply(len))

156133

In [ ]:
$0.0600 / 1K tokens

In [9]:
df.shape

(1000, 13)

In [10]:
1000*50

50000

In [ ]:
1627921

In [11]:
(((156133+50000)*1000/750)/1000)*0.0600

16.49064

## requests

In [8]:
df=df.loc[df.label==1]
df=df.loc[(~df.task_annotation.isna())|(~df.method_annotation.isna())].reset_index(drop=True)

In [9]:
df.shape

(130, 21)

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['method_prompt_text']
    print(input_prompt)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("###################################")

In [ ]:
for i,d in df.iterrows():
    input_prompt=d['method_prompt_text']
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=40,logprobs=1)
    df.loc[i,'GPT3_response']=completion.choices[0].text
    f = open("/cluster/scratch/fgonzalez/logs_gpt3/responses_m2f.txt", "a")
    f.write(d['method_prompt_text'])
    f.write(completion.choices[0].text)
    f.write("################################")
    f.write("\n")
    f.close()
    print(input_prompt)
    print(completion.choices[0].text)
    print("Gold tasks: ",d['task_annotation'])
    print("Gold methods: ",d['method_annotation'])
    print("################################")

In [12]:
df=df.assign(clean_response=df.GPT3_response.replace("\n","",regex=True))
df=df.assign(clean_response=df.clean_response.replace("\[method2\]",",",regex=True).replace("\[method3\]",",",regex=True).replace("method1\]","",regex=True).replace("1\]","",regex=True).replace("\[3\]",",",regex=True).replace("\[2\]",",",regex=True).str.rstrip(']'))
df=df.assign(clean_response=df.clean_response.str.split(","))

methods=df.loc[:,['ID','clean_response']]

methods=methods.explode("clean_response")

methods=methods.rename(columns={'clean_response':'methods'})

methods=methods.assign(methods=methods.methods.str.lstrip().str.rstrip())
methods=methods.loc[methods.methods!='']

In [16]:
methods

,ID,methods
0,wandji-tchami-grabar-2014-towards,semantic annotation
0,wandji-tchami-grabar-2014-towards,contrastive analysis
0,wandji-tchami-grabar-2014-towards,medical terminology
1,soh-etal-2019-legal,topic modeling
1,soh-etal-2019-legal,word embedding
...,...,...
128,khanna-etal-2022-idiap,Attention Mechanism
128,khanna-etal-2022-idiap,Contextualized BERT
129,solorio-etal-2013-case,authorship attribution
129,solorio-etal-2013-case,voting scheme


In [17]:
df.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_method2_f.csv",index=False)

methods.to_csv(output_path+"sg_ie/gpt3/GPT3_responses_method2_cleaned_f.csv",index=False)